In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.60'

In [3]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR5868581_2.len10.assembly.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/TaiwaneseOolong.len10.pkl"

In [4]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [5]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,TASGRWNNAGKSEEHTSELQSRSDLVCRLLLEKKRGGGG,39
1,1,MHIGKIEEHTSELQSRSDLVCRHQLEKKNGGG,32
2,2,MAVLSWWSDLSG,14
3,3,PGVRFIPHRQVCLPKMAHLELSIPWRGSTEQPRRP,35
4,4,TTLGQTEEHTSELQSRSDLVCRLQLEKKKGG,31
...,...,...,...
2719095,2719095,SEEHTSELQSRSYLVSRLLLEKKKEG,26
2719096,2719096,RAVGHRNPASGSSRIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,45
2719097,2719097,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34
2719098,2719098,PKMAHLELSIPWRGSTEQPRRPTYLKFENRSSALRPRCL,40


In [ ]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

Remove Duplicate

In [ ]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

In [ ]:
df.to_pickle(plk)
tmp_df = pd.read_pickle(plk)
tmp_df